In [8]:
#!pip install datasets
from datasets import load_dataset

#import warnings
#warnings.filterwarnings("ignore")

import pandas as pd
pd.plotting.register_matplotlib_converters()
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
print("Setup Complete")
from sklearn.metrics import auc, roc_curve, precision_score, recall_score, precision_recall_curve

import optuna
from sklearn.preprocessing import TargetEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import log_loss
from xgboost import XGBClassifier


import numpy as np

Error importing huggingface_hub.hf_api: cannot import name 'XetAuthorizationError' from 'huggingface_hub.errors' (/Users/mounir/Desktop/PROJETS/Statapp/.venv/lib/python3.12/site-packages/huggingface_hub/errors.py)


ImportError: cannot import name 'XetAuthorizationError' from 'huggingface_hub.errors' (/Users/mounir/Desktop/PROJETS/Statapp/.venv/lib/python3.12/site-packages/huggingface_hub/errors.py)

In [13]:
ds = load_dataset("criteo/FairJob")
df = ds['train'].to_pandas()

In [15]:
from sklearn.metrics import log_loss

def print_results(X_test, y_test, model):
    # Calcul des probabilités des prédictions
    y_scores = model.predict_proba(X_test)[:, 1]  # Probabilité d'être classe 1

    # Calcul des courbes précision-rappel
    precisions, recalls, thresholds = precision_recall_curve(y_test, y_scores)

    # Calcul du F1-score pour chaque seuil
    f1_scores = 2 * (precisions * recalls) / (precisions + recalls + 1e-9)  # Évite division par zéro

    # Trouver l'index du meilleur F1-score
    best_threshold_index = np.argmax(f1_scores)

    # Le tableau "thresholds" est plus court d'un élément → il faut ajuster l'indexation
    if best_threshold_index == 0:  # Si le max est au début, on prend le premier seuil
        best_threshold = thresholds[0]
    else:
        best_threshold = thresholds[best_threshold_index - 1]

    # Précision et rappel au meilleur seuil
    precision_optimal = precisions[best_threshold_index]
    recall_optimal = recalls[best_threshold_index]
    f1_score_optimal = f1_scores[best_threshold_index]

    # Calcul du log loss
    log_loss_value = log_loss(y_test, y_scores)

    print(f'Best Threshold: {best_threshold}')
    print(f'\nPrecision at Best Threshold: {precision_optimal}')
    print(f'Recall at Best Threshold: {recall_optimal}')
    print(f'F1 Score at Best Threshold: {f1_score_optimal}')
    print(f'Log Loss: {log_loss_value}')

    plt.figure(figsize=(8, 6))
    plt.plot(thresholds, precisions[1:], label='Précision', color='blue')
    plt.plot(thresholds, recalls[1:], label='Recall', color='red')
    plt.axvline(x=best_threshold, color='green', linestyle='--', label="Seuil Optimal")
    plt.xlabel("Seuil de classification")
    plt.ylabel("Score")
    plt.title("Courbe Précision-Recall en fonction du seuil")
    plt.legend()
    plt.grid()
    plt.show()


In [16]:
from sklearn.metrics import log_loss
import numpy as np

def evaluate_log_loss_by_class(y_true, y_proba):
    """
    Calcule la log loss globale, la log loss par classe, et la confiance moyenne sur les vrais positifs.
    
    Parameters
    ----------
    y_true : array-like
        Vecteur des vraies étiquettes (0 ou 1)
    y_proba : array-like, shape (n_samples, 2)
        Probabilités prédites (sorties de predict_proba)

    Returns
    -------
    dict with:
        - 'log_loss_global'
        - 'log_loss_y0'
        - 'log_loss_y1'
        - 'mean_proba_y1' (moyenne des p(y=1) quand y=1)
    """
    y_true = np.array(y_true)
    y_proba = np.array(y_proba)

    return {
        "log_loss_global": round(log_loss(y_true, y_proba), 5),
        "log_loss_y0": round(log_loss(y_true[y_true == 0], y_proba[y_true == 0], labels=[0, 1]), 5),
        "log_loss_y1": round(log_loss(y_true[y_true == 1], y_proba[y_true == 1], labels=[0, 1]), 5),
        "mean_proba_y1": round(float(y_proba[y_true == 1, 1].mean()), 5)
        }



In [17]:
from sklearn.model_selection import train_test_split

X = df.drop(columns=['click','user_id','impression_id','product_id','protected_attribute'])
y = df['click']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [18]:
from sklearn.preprocessing import TargetEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import log_loss
from xgboost import XGBClassifier


xgb_tree_method = "hist"

cat_cols_bin = list(X_train.columns[0:3])
rank_col = ['rank']
cat_cols = list(X_train.columns[4:17])

data_preproc = ColumnTransformer([('multicat_encoding', TargetEncoder(target_type='binary'), cat_cols)], 
                              remainder='passthrough', force_int_remainder_cols=False)

xgb_pipe = Pipeline([('preprocessing', data_preproc),
                     ('xgb',XGBClassifier())])

In [1]:
scale_pos_weight = (
        (np.sum(y_train < 1) / np.sum(y_train > 0))
    )

def objective(trial):
    X_train_train, X_val, y_train_train, y_val = train_test_split(
        X_train, y_train, test_size=0.2, stratify=y_train
    )

    max_depth = trial.suggest_int("max_depth", 3, 10)
    min_child_weight = trial.suggest_float("min_child_weight", 0.0001, 100, log=True)
    subsample = trial.suggest_float("subsample", 0.5, 1)
    learning_rate = trial.suggest_float("learning_rate", 0.001, 1, log=True)
    colsample_bytree = trial.suggest_float("colsample_bytree", 0.5, 1)
    reg_lambda = trial.suggest_float("reg_lambda", 0.1, 10, log=True)
    gamma = trial.suggest_float("gamma", 0.001, 100, log=True)
    base_score = trial.suggest_float("base_score", 1e-4, 1 - 1e-4)

    
    xgb_params = {
        'xgb__max_depth': max_depth,
        'xgb__min_child_weight': min_child_weight,
        'xgb__subsample': subsample,
        'xgb__learning_rate': learning_rate,
        'xgb__colsample_bytree': colsample_bytree,
        'xgb__reg_lambda': reg_lambda,
        'xgb__gamma': gamma,
        'xgb__base_score': base_score,
        'xgb__scale_pos_weight': scale_pos_weight,
    }
    
    xgb_pipe.set_params(**xgb_params)
    xgb_pipe.fit(X_train_train, y_train_train)

    prob_val = xgb_pipe.predict_proba(X_val)
    
    return log_loss(y_val, prob_val)

n_trials = 2 # Increase to ~100

study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=n_trials)
optimal_no_scale = study.best_trial

print("\nOptimization done")

xgb_tuned_params = {
    'xgb__max_depth': optimal_no_scale.params["max_depth"],
    'xgb__min_child_weight': optimal_no_scale.params["min_child_weight"],
    'xgb__gamma': optimal_no_scale.params["gamma"],
    'xgb__subsample': optimal_no_scale.params["subsample"],
    'xgb__learning_rate': optimal_no_scale.params["learning_rate"],
    'xgb__colsample_bytree': optimal_no_scale.params["colsample_bytree"],
    'xgb__reg_lambda': optimal_no_scale.params["reg_lambda"],
    'xgb__base_score': optimal_no_scale.params["base_score"],
    'xgb__scale_pos_weight': scale_pos_weight,
}
    
xgb_pipe.set_params(**xgb_tuned_params)
xgb_pipe.fit(X_train, y_train)

print("\nEntrainement terminé")

xgb_adapted_scale_tuned = evaluate_log_loss_by_class(y_test, xgb_pipe.predict_proba(X_test))

NameError: name 'np' is not defined